In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

import sscCdi
from sscCdi import unwrap_in_parallel, equalize_frames_parallel, make_bad_frame_null
import sscRadon

%matplotlib widget

In [ ]:
# path = "/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/proc/recons/glass_sphere/2023-07-28-17h04m_glass_sphere.hdf5" 
path = "/ibira/lnls/labs/tepui/home/yuri.tonin/00000000/proc/recons/glass_sphere/2023-05-16-15h38m_glass_sphere.hdf5" 
data = h5py.File(path,'r')['recon/object'][()]
angles = h5py.File(path,'r')['recon/angles'][()][:,0::2]
data=data[:,1350:-1300,1200:-1400]


# path = "/ibira/lnls/beamlines/caterete/proposals/20221798/proc/recons/sorgoBMR08_mem1_FAA_YuriTest/YuriTest_sorgoBMR08_mem1_FAA.hdf5"
# data = h5py.File(path,'r')['recon/object'][()]
# angles = h5py.File(path,'r')['recon/angles'][()][:,0::2]
# data=data[:,1345:3040,1280:2980]


# path = "/ibira/lnls/beamlines/caterete/proposals/20221798/proc/recons/sorgoWT05_mem2_FAA_01/2023-07-25-14h54m_sorgoWT05_mem2_FAA_01.hdf5"
# data = h5py.File(path,'r')['recon/object'][()]
# angles = h5py.File(path,'r')['recon/angles'][()][:,0::2]
# data=data[:,1300:-1500,1300:-1500]
# data.shape



In [ ]:
from sscCdi import sort_angles
from sscCdi import reorder_slices_low_to_high_angle

sorted_angles = sort_angles(angles) # input colums with frame number and angle in rad
data = reorder_slices_low_to_high_angle(data, sorted_angles)

In [ ]:
sscCdi.misc.deploy_visualizer(np.angle(data),title='sample',cmap='gray',axis=0)

In [ ]:
dic={}
dic["bad_frames_before_equalization"] = []

dic["CPUs"] = 64
dic["equalize_invert"] = True # invert phase shift signal from negative to positive
dic["equalize_ROI"] = [0,1550,1500,1600] # region of interest of null region around the sample used for phase ramp and offset corrections
dic["equalize_remove_phase_gradient"] = True  # if empty and equalize_ROI = [], will subtract best plane fit from whole image
dic["equalize_set_min_max"]= [] # [minimum,maximum] threshold values for whole volume
dic["equalize_non_negative"] = True # turn any remaining negative values to zero
dic["equalize_local_offset"] = False # remove offset of each frame from the mean of ROI 

In [ ]:
unwrapped_sinogram = unwrap_in_parallel(np.angle(data))

In [ ]:
equalized_sinogram = equalize_frames_parallel(unwrapped_sinogram,dic)
# equalized_sinogram = np.where(equalized_sinogram<-0.6,0,equalized_sinogram)
sscCdi.misc.deploy_visualizer(equalized_sinogram,title='sample',cmap='gray')

In [ ]:
sscCdi.misc.deploy_visualizer(equalized_sinogram[3::],title='sample',cmap='gray')

In [ ]:
from sscCdi import plot_volume_histogram

plot_volume_histogram(equalized_sinogram)

In [ ]:
dic["CPUs"] = 32 
dic["wiggle_reference_frame"] = 50

temp_tomogram, shift_vertical = sscRadon.radon.get_wiggle( equalized_sinogram, "vertical", dic["CPUs"], dic["wiggle_reference_frame"] )
# temp_tomogram, shift_vertical = sscRadon.radon.get_wiggle( temp_tomogram, "vertical", dic["CPUs"], dic["wiggle_reference_frame"] )

In [ ]:
sscCdi.misc.deploy_visualizer(temp_tomogram,title='sample',cmap='gray',axis=0)

In [ ]:
aligned_sinogram, shift_horizontal, wiggle_cmas_temp = sscRadon.radon.get_wiggle( temp_tomogram, "horizontal", dic["CPUs"], dic["wiggle_reference_frame"] )

In [ ]:
sscCdi.misc.deploy_visualizer(temp_tomogram,title='sample',cmap='gray',axis=1)

In [ ]:
sscCdi.misc.deploy_visualizer(aligned_sinogram,title='sample',cmap='gray',axis=1)

In [ ]:
sscCdi.misc.deploy_visualizer(temp_tomogram,title='sample',cmap='gray',axis=2)

In [ ]:
sscCdi.misc.deploy_visualizer(aligned_sinogram,title='sample',cmap='gray',axis=2)

## Cross Correlation of Variance Field 

In [ ]:
sscCdi.misc.deploy_visualizer(np.angle(data),title='sample',cmap='gray',axis=0)

In [ ]:
cropped_aligned_volume = alignment_variance_field(data, fft_upsampling=10,return_common_valid_region=True, remove_null_borders = True)

# idx_to_delete = [28,30,45,49,65,66,91,113]
# cropped_aligned_volume = np.delete(cropped_aligned_volume,idx_to_delete,axis=0)
# sscCdi.misc.deploy_visualizer(np.angle(cropped_aligned_volume),title='sample',cmap='gray')

In [ ]:
unwrapped_sinogram = unwrap_in_parallel(np.angle(cropped_aligned_volume))

In [ ]:
equalized_sinogram = equalize_frames_parallel(unwrapped_sinogram,dic["equalize_invert"],dic["equalize_gradient"],dic["equalize_outliers"],dic["equalize_global_offset"], dic["equalize_local_offset"])

In [ ]:
sscCdi.misc.deploy_visualizer(equalized_sinogram[10:500],title='sample',cmap='gray')

In [ ]:
sscCdi.misc.deploy_visualizer(equalized_sinogram,title='sample',cmap='gray',axis=1)

In [ ]:
def plot_volume_histogram(volume,bins=100):
    
    maximum = np.max(volume)
    minimum = np.min(volume)
    mean    = np.mean(volume)
    stddev  = np.std(volume)
    
    fig, ax = plt.subplots()
    ax.hist(volume.flatten(),bins=bins)
    ax.grid()
    ax.set_title(f'Max={maximum:.2f}   Min={minimum:.2f}   Mean={mean:.2f}   StdDev={stddev:.2f}')
    
plot_volume_histogram(equalized_sinogram)

In [ ]:
sscCdi.misc.deploy_visualizer(equalized_sinogram,title='sample',cmap='gray',axis=2)

In [ ]:
def plot_sinogram_slices(sinogram,cmap='gray'):
    shape = sinogram.shape
    fig, ax = plt.subplots(1,2,figsize=(20,5))
    ax[0].imshow(sinogram[:,shape[1]//2,:],cmap=cmap)
    ax[1].imshow(sinogram[:,:,shape[2]//2],cmap=cmap)    
    
plot_sinogram_slices(equalized_sinogram)

## Vertical mass fluctuation

In [ ]:
aligned_volume2 = alignment_vertical_mass_fluctuation(cropped_aligned_volume, use_phase_gradient = False, plot = True)

In [ ]:
sscCdi.misc.deploy_visualizer(np.angle(aligned_volume2),title='sample',cmap='gray')

In [ ]:
sscCdi.misc.deploy_visualizer(np.angle(aligned_volume2),title='sample',cmap='gray',axis=2)

In [ ]:
sscCdi.misc.deploy_visualizer(np.angle(cropped_aligned_volume),title='sample',cmap='gray',axis=2)

In [ ]:
unwrapped_volume = unwrap_in_parallel(np.angle(aligned_volume2))
unwrapped_volume = unwrapped_volume[:,100::,:]

sscCdi.misc.deploy_visualizer(unwrapped_volume,title='sample',cmap='gray')

In [ ]:
dic = {}
dic["bad_frames_before_unwrap"] = []#[28,30,45,49,65,66,91,113]
dic["equalize_gradient"] = [True,[1400,1500,0,1500],0]
dic["equalize_global_offset"] = False
dic["equalize_invert"] = True
dic["equalize_local_offset"] = [1400,1500,0,1500]
dic["equalize_outliers"] = 0

# equalized_sinogram = equalize_frames_parallel(unwrapped_volume,dic["equalize_invert"],dic["equalize_gradient"],dic["equalize_outliers"],dic["equalize_global_offset"], dic["equalize_local_offset"])
# sscCdi.misc.deploy_visualizer(equalized_sinogram,title='sample',cmap='gray')

In [ ]:
sscCdi.misc.deploy_visualizer(equalized_sinogram,title='sample',cmap='gray',axis=2)

In [ ]:
sscCdi.misc.deploy_visualizer(equalized_sinogram,title='sample',cmap='gray',axis=1)